# ÁLGEBRA LINEAR COMPUTACIONAL

## SEGUNDA AVALIAÇÃO - 2020.2 - PROFESSOR ERITO MARQUES   

---
### GRUPO:
**Ícaro Santos Barcelos Pereira - _2017780187_**\
**Leandro Bataglia Pereira - _2018780199_**\
**Pedro Henrique Farolfi Camelo - _2018780326_**

---

Repositório dos algoritmos solicitados na segunda prova da disciplina de Álgebra Linear Computacional.
##### Instruções:
As respostas devem ser enviadas no formato jupyter notebook.\
Os trabalhos devem ser feitos em grupo. Não é permitida troca de informações entre os grupos.\
Fixada a data de entrega, não serão aceitos trabalhos entregues fora dessa data.

In [1]:
import numpy as np
import pandas as pd
import normas
from cholesky import decomposicao_cholesky
from gauss import eliminacao_gauss
from fatoracaoLU import fatoracaoLU
from sympy import Matrix, init_printing
init_printing()
#BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*)

In [2]:
# Coverti isso aqui em um procedimento, a parada é que vc pode mandar isso 
# ocorrer no início de qualquer célula e pode alterar o excel em tempo de execução :)

# CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64'
def ler():
    df_matriz = pd.read_excel(r"dados/matriz_A.xlsx", header=None) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor = pd.read_excel(r"dados/vetor_b.xlsx", header=None) # Le o arquivo do excel que contém o vetor b sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor

### 1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie:
**a)** se A é tridiagonal;

In [3]:
def verifica_tridiagonal(matriz_A):
    linhas, colunas = np.shape(matriz_A)

    for i in range(linhas):
        for j in range(colunas):
            if np.abs(i-j) <= 1 and matriz_A[i,j] == 0:
                return False
            if np.abs(i-j) >= 2 and matriz_A[i,j] != 0:
                return False
    return True

In [4]:
matriz, vetor = ler()
verifica_tridiagonal(matriz)

False

**b)** se A é triangular superior;

In [5]:
def verifica_tri_superior(matriz_A):
    linhas = len(matriz_A)

    for i in range(linhas):
        for j in range(i):
            if matriz_A[i,j] != 0:
                return False
    return True

In [6]:
matriz, vetor = ler()
verifica_tri_superior(matriz)

False

**c)** se A é triangular inferior;

In [7]:
def verifica_tri_inferior(matriz_A):
    colunas = len(matriz_A[0])

    for j in range(colunas):
        for i in range(j):
            if matriz_A[i,j] != 0:
                return False
    return True

In [8]:
matriz, vetor = ler()
verifica_tri_inferior(matriz)

False

**d)** se Traço de A é maior que 4.

In [9]:
def traco(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    n, traco = min((linhas, colunas)), 0

    for i in range(n):
        traco += matriz_A[i,i]
    if traco > 4:
        print("O traço da matriz é maior que 4")
        return True
    print("O traço da matriz não é maior que 4")
    return False

In [10]:
matriz, vetor = ler()
traco(matriz)

O traço da matriz é maior que 4


True

---
### 2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

In [11]:
# TESTE DO ALGORITMO DE DECOMPOSIÇÃO DE CHOLESKY E NORMA RESIDUAL (cholesky.py, normas.py) '''
matriz, vetor = ler()
delta_x = decomposicao_cholesky(matriz, vetor)

print("Vetor x resultado da Decomposição de Cholesky:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)

Vetor x resultado da Decomposição de Cholesky:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 

Vetor residual:
 [-4.36557457e-11  1.45519152e-11 -2.91038305e-11 -3.63797881e-11
 -2.91038305e-11  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -7.27595761e-12  1.45519152e-11  0.00000000e+00  0.00000000e+00
  2.91038305e-11 -7.27595761e-12 -2.18278728e-11 -3.27418093e-11
  4.36557457e-11  1.45519152e-11  7.27595761e-12 -4.00177669e-11
  0.00000000e+00  2.91038305e-11]
Resíduo da solução x: 4.1378398779635326e-16
A solução encontrada é precisa.


In [12]:
# TESTE DO ALGORITMO DE ELIMINAÇÃO DE GAUSS E NORMA RESIDUAL (gauss.py, normas.py) '''
delta_x = eliminacao_gauss(matriz, vetor)

print("Vetor x resultado da Eliminação de Gauss:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)
print("\n")

A matriz não é triangular
Vetor x resultado da Eliminação de Gauss:
 <class 'RuntimeError'> 





In [13]:
# TESTE DO ALGORITMO DE FATORAÇÃO LU E NORMA RESIDUAL (fatoracaoLU.py, normas.py)'''
delta_x = fatoracaoLU(matriz,vetor)

print("Vetor x resultado da Fatoração LU:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)

A matriz não é triangular
Vetor x resultado da Fatoração LU:
 <class 'RuntimeError'> 



ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)